In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import gc
import timeit
from Bio import SeqIO
from joblib import dump, load
import multiprocessing as mp
import time
%matplotlib inline
import pickle
from tqdm.auto import tqdm
import ipywidgets
tqdm.pandas()
from ipywidgets import IntProgress
! jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


## Step 1 Generate kmers of size k

## !!! This code is run once. In step 2 load processed data written on disk !!!

## splits = 6  works well for this data and 16gb RAM

## For smaller memory size, Higher number of splits may be needed. 

In [ ]:
###############################################################################
#        Conversion step 1: Extract kmers from DNA into a list of kmers       #
###############################################################################

k=10

split = 6
columnName = str(k)+"mers"
gc.enable()
# gc.disable()
###############################################################################
#                            Read fasta sequences                              #
###############################################################################

# 
from Bio import SeqIO
#chromid = []
chromseq = []
for sample in SeqIO.parse('D:\DataSet\Data\sequences.fasta', "fasta"):
    #chromid.append(sample.id)
    chromseq.append(str(sample.seq))


###############################################################################
#             function that extracts kmers of size k from a sequence          #
###############################################################################
def getKmers(sequence, size=k):
# Scan the sequence with a window of the given size and add
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

###############################################################################
#      Split the data in multiple splits to reduce RAM-DISK hard faults       #
###############################################################################
splits =np.array_split(pd.DataFrame(chromseq,columns=['seq']), split)


###############################################################################
#     For each split, extract kmers   & write result to disk in chunks        #
###############################################################################
start = time.time()
for i in range(split):

    DataFrame(splits[i].progress_apply(lambda x: getKmers(x['seq']), axis=1), columns=[columnName]).reset_index().to_feather(r"D:\DataSet\LSTMonBoW\k"+str(k)+"-part-"+str(i)+"-kmer.feather")
    #free memory
    gc.collect()
    end = time.time()
#     total time taken
print(f"Runtime is {end - start} secs")

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [5]:
# !jupyter nbconvert --to webpdf --allow-chromium-download "Step 1 CV Pre-processing .ipynb"